# Walmart Tech. P13N - ML code interview

In [1]:
# Recommender System: Matrix Factorization, Neural Collaborative Filtering, Session-based recommender system
# ** Written by Jonogseok Han (EE in P13N, email - work: jongseok.han0@walmart.com / personal: jshan.cse@gmail.com)

In [2]:
import numpy as np
import torch
import torch.nn as nn

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Section 0: **Data Parsing & Preprocessing**

In [5]:
from pathlib import Path
train_url = "https://raw.githubusercontent.com/Jongseok-han/ml_recsys/main/interview_note/data/train.csv"
test_url = "https://raw.githubusercontent.com/Jongseok-han/ml_recsys/main/interview_note/data/test.csv"

path_data = Path("data")
path_data.mkdir(exist_ok=True)

path_train = path_data/"train.csv"
path_test = path_data/"test.csv"

from urllib.request import urlretrieve
if not path_train.exists(): urlretrieve(train_url, path_train)
if not path_test.exists(): urlretrieve(test_url, path_test)

### Data Description: **[MovieLens](https://files.grouplens.org/datasets/movielens/ml-latest-small-README.html)**

MovieLens dataset (ml-latest-small) - we offer two files `train.csv` and `test.csv`. Each line of these files represents one rating of one movie by one user, and has the following format:

    userId,movieId,rating,timestamp

Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars). We will normalize the ratings in (0.1 - 1.0) scale

In [6]:
def load_dataset (filename):
  input_data = []
  with open (filename) as fin:
    for i, line in enumerate (fin):
      parts = line.strip().split (',')
      input_data.append([parts[0],parts[1],float(parts[2])/5, parts[3]])
  return np.array(input_data).astype(str)

In [7]:
# data statistics
train_data = load_dataset ("./data/train.csv")
test_data = load_dataset ("./data/test.csv")
train_size, test_size = len(train_data), len(test_data)

print ("=== Dataset statistics ===")
print ("Number of Train Data: {} / Number of Test Data: {}".format(train_size, test_size))
print ("Min/max Ratings of Train Data: ({}, {})".format(min(train_data[:,2]), max(train_data[:,2])))

=== Dataset statistics ===
Number of Train Data: 90718 / Number of Test Data: 9715
Min/max Ratings of Train Data: (0.1, 1.0)


In [8]:
# sort userID, itemID in ascending order: userID from 0 to N, itemID from 0 to M
original_data = np.vstack((train_data, test_data))

unique_users = sorted(list(set(original_data[:, 0])))
unique_items = sorted(list(set(original_data[:, 1])))
num_users, num_items = len(unique_users), len(unique_items)

user_dic = {user:idx for (idx,user) in enumerate(unique_users)}
item_dic = {item:idx for (idx,item) in enumerate(unique_items)}

for (idx, row) in enumerate(original_data):
    user,item = user_dic[row[0]],item_dic[row[1]]
    original_data[idx,0],original_data[idx,1] = int(user),int(item)

## Section 1: **Matrix Factorization**

The matrix factorization algorithm works by decomposing the rating matrix into the product of two lower dimensionality rectangular matrices. Let’s define $K$ as the number of latent dimensions. Then, we learn a user embedding matrix $U \in  \mathbb{R}^{N \times K}$ and an item embedding matrix $V \in  \mathbb{R}^{M \times K}$ ($N$ and $M$ are the number of users and items, respectively).
We will approximate a rating $R_{u,i}$ by updating $U$ and $V$ matrices.  
$$R_{u,i} \approx \sum_{k=1}^{K} U_{u,k} V_{i,k}$$

<img src='https://drive.google.com/uc?export=view&id=1qZyCdjsZ_8hlfIxgPbO1Ht31gqj14LVl'>

In [9]:
# build rating matrix before Matrix Factorization
gt_matrix = np.zeros((num_users, num_items))

for interaction in original_data:
  gt_matrix[int(interaction[0]),int(interaction[1])] = float(interaction[2])

print("User-Item Rating Matrix - shape:{}\n\n{}".format(gt_matrix.shape, gt_matrix))

User-Item Rating Matrix - shape:(610, 9338)

[[0.8 0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.9 0.  0.  ... 0.  0.  0. ]
 [0.  0.8 0.  ... 0.  0.  0. ]]


In [10]:
# define train & test set. We ignore a timestamp column since MF didn't consider sequential information
train_data, test_data = original_data[:train_size,:3], original_data[train_size:,:3]

In [11]:
# class MF(nn.Module):
#     def __init__(self, num_users, num_items, latent_dim):
#         torch.manual_seed(0)
#         np.random.seed(0)
#         super(MF, self).__init__()

#         #### YOUR CODE GOES HERE ####

#         #############################

#     def forward(self, input):

#         #### YOUR CODE GOES HERE ####

#         #############################

In [13]:
latent_dim = 10
mf_model = MF(num_users = num_users, num_items=num_items, latent_dim = latent_dim).to(device)

In [14]:
# loss function & experimental setting
# criterion = None
optimizer = torch.optim.Adam(mf_model.parameters())

batch_size = 1024
batch_num = len(train_data)//batch_size + 1
max_epoch = 500
patience = 10

In [15]:
min_RMSE, best_epoch = np.inf, 0

for epoch in range(max_epoch):

  # batch training
  mf_model.train()
  train_loss = 0
  for batch in range(batch_num):
    std, end = batch*batch_size, (batch+1)*batch_size
    end = len(train_data) if (batch+1)*batch_size > len(train_data) else (batch+1)*batch_size
    input, gt = train_data[std:end,:2], train_data[std:end,-1]

    optimizer.zero_grad()
    input = torch.LongTensor(input.astype(int)).to(device)
    gt = torch.FloatTensor(gt.astype(float)).to(device)

    pred = mf_model(input)
    loss = criterion(pred,gt)
    train_loss += loss

    loss.backward()
    optimizer.step()

  # test evaluation
  mf_model.eval()
  test_input, test_gt = test_data[:,:2], test_data[:,-1]
  test_gt = torch.FloatTensor(test_gt.astype(float))

  test_pred = mf_model(torch.LongTensor(test_input.astype(int)).to(device))
  test_RMSE = ((test_pred.detach().cpu()-test_gt)**2).sqrt().sum()/test_size

  if epoch % 10==0:
    print("epoch {} \ttrain_loss = {:.4f}\ttest_RMSE = {:.4f}".format(epoch,train_loss.item(),test_RMSE))

  if test_RMSE < min_RMSE:
    min_RMSE = test_RMSE
    best_epoch = epoch
  elif epoch - best_epoch >= patience:
    print("\nStop Training at epoch {} - test_RMSE = {:.4f}".format(epoch, test_RMSE))
    break


epoch 0 	train_loss = 298.9193	test_RMSE = 1.5540
epoch 10 	train_loss = 43.3422	test_RMSE = 0.4986
epoch 20 	train_loss = 16.1494	test_RMSE = 0.3207
epoch 30 	train_loss = 8.6717	test_RMSE = 0.2595
epoch 40 	train_loss = 5.3630	test_RMSE = 0.2248
epoch 50 	train_loss = 3.6059	test_RMSE = 0.2015
epoch 60 	train_loss = 2.6309	test_RMSE = 0.1859
epoch 70 	train_loss = 2.0989	test_RMSE = 0.1767
epoch 80 	train_loss = 1.8099	test_RMSE = 0.1723
epoch 90 	train_loss = 1.6392	test_RMSE = 0.1706
epoch 100 	train_loss = 1.5201	test_RMSE = 0.1704

Stop Training at epoch 107 - test_RMSE = 0.1707


## Section 2: **Neural Collaborative Filtering**

For NCF, we will implement a simple multi-layer perceptron (MLP) model as below.

<img src='https://drive.google.com/uc?id=1kEFTCa7ragS44aYmF89V958TbJpcPv-u'>

The embedding layer transforms user_id and item_id to user and item latent vectors, respectively. Here, we will use U, V instead of nn.Embedding in order to focus on the performace from NCF layers.

For NCF layers, please use the size of [64,512] and [512,512] and ReLU activation function. The output layer is also fully-connected (with ReLU activation) with size of [512,1] to produce a single prediction rating


In [ ]:
# class NCF(nn.Module):
#     def __init__(self, num_users, num_items, latent_dim):
#         torch.manual_seed(0)
#         np.random.seed(0)
#         super(NCF, self).__init__()

#         #### YOUR CODE GOES HERE ####

#         #############################

#     def forward(self, input):

#         #### YOUR CODE GOES HERE ####

#         #############################

In [21]:
ncf_model = NCF(num_users, num_items, latent_dim).to(device)
print(ncf_model)

NCF(
  (mlp): Sequential(
    (0): Linear(in_features=20, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1, bias=True)
  )
)


In [22]:
criterion = None
optimizer = torch.optim.Adam(ncf_model.parameters(), lr=1e-4)

In [23]:
batch_size = 1024
batch_num = len(train_data)//batch_size + 1
max_epoch = 500
patience = 10

In [24]:
min_RMSE, best_epoch = np.inf, 0

for epoch in range(max_epoch):

  # batch training
  ncf_model.train()
  train_loss = 0
  for batch in range(batch_num):
    std, end = batch*batch_size, (batch+1)*batch_size
    end = len(train_data) if (batch+1)*batch_size > len(train_data) else (batch+1)*batch_size
    input, gt = train_data[std:end,:2], train_data[std:end,-1]

    optimizer.zero_grad()
    input = torch.LongTensor(input.astype(int)).to(device)
    gt = torch.FloatTensor(gt.astype(float)).to(device)

    pred = ncf_model(input)
    loss = criterion(pred,gt)
    train_loss += loss

    loss.backward()
    optimizer.step()

  # test evaluation
  ncf_model.eval()
  test_input, test_gt = test_data[:,:2], test_data[:,-1]
  test_gt = torch.FloatTensor(test_gt.astype(float))

  test_pred = ncf_model(torch.LongTensor(test_input.astype(int)).to(device))
  test_RMSE = ((test_pred.detach().cpu()-test_gt)**2).sqrt().sum()/test_size

  if epoch % 10==0:
    print("epoch {} \ttrain_loss = {:.4f}\ttest_RMSE = {:.4f}".format(epoch,train_loss.item(),test_RMSE))

  if test_RMSE < min_RMSE:
    min_RMSE = test_RMSE
    best_epoch = epoch
  elif epoch - best_epoch >= patience:
    print("\nStop Training at epoch {} - test_RMSE = {:.4f}".format(epoch, test_RMSE))
    break


epoch 0 	train_loss = 6.8992	test_RMSE = 0.1784
epoch 10 	train_loss = 3.7474	test_RMSE = 0.1660
epoch 20 	train_loss = 3.3262	test_RMSE = 0.1527
epoch 30 	train_loss = 3.0077	test_RMSE = 0.1458
epoch 40 	train_loss = 2.7876	test_RMSE = 0.1436
epoch 50 	train_loss = 2.6126	test_RMSE = 0.1398
epoch 60 	train_loss = 2.4437	test_RMSE = 0.1380
epoch 70 	train_loss = 2.3195	test_RMSE = 0.1372

Stop Training at epoch 78 - test_RMSE = 0.1377


## Section 3: **Session-based Recommender System**

Session-based recommender system is a type of recommendation engine that generates personalized suggestions for users based on the actions they have taken within a session. These models use algorithms to analyze the **sequence of interactions within a session**, such as pages viewed, items added to a cart, or search queries, to predict and recommend **next item** that the user is likely to be interested in.

For example, if a customer spends a session looking at various clothes, shoes and accessories, the recommender system might suggest related products like sneakers leveraging insights from the current session to enhance the shopping experience.
<img src='https://drive.google.com/uc?export=view&id=1UH58CB1ENbngSWpGx9agqN4w2WBrFoIk'>

In [ ]:
original_data

array([['0', '0', '0.8'],
       ['0', '3536', '0.8'],
       ['0', '6636', '0.8'],
       ...,
       ['568', '1590', '0.8'],
       ['568', '1676', '0.6'],
       ['568', '1720', '0.7']], dtype='<U32')

In [ ]:
# only consider positive interaction for sequential recommendation
positive_signal = 0.5
positive_interactions = original_data[(np.array([float(rating) for rating in original_data[:,2]]) > positive_signal)]

In [ ]:
# re-sort userID, itemID in ascending order: userID from 0 to N, itemID from 0 to M
unique_pos_users = sorted(list(set(positive_interactions[:, 0])))
unique_pos_items = sorted(list(set(positive_interactions[:, 1])))
num_items = len(unique_pos_items)

pos_user_dic = {user:idx for (idx,user) in enumerate(unique_pos_users)}
pos_item_dic = {item:idx for (idx,item) in enumerate(unique_pos_items)}

for (idx, row) in enumerate(positive_interactions):
    user, item = pos_user_dic[row[0]], pos_item_dic[row[1]]
    positive_interactions[idx,0], positive_interactions[idx,1] = int(user),int(item)

In [ ]:
def train_test_split_user(data, test_ratio = 0.1):
    (users,counts) = np.unique(data[:,0], return_counts = True)
    users = users[counts>=10]

    user_dic = {int(user):idx for (idx,user) in enumerate(users)}
    new_data = []
    for i in range(len(data)):
        if int(data[i,0]) in user_dic:
            new_data.append([int(data[i,0]),int(data[i,1]),0])

    new_data = np.array(new_data)
    sequence_dic = {int(user):[] for user in user_dic.keys()}

    for i in range(len(new_data)):
        sequence_dic[int(new_data[i,0])].append(i)

    for user in sequence_dic.keys():
        cur_test = int(test_ratio * len(sequence_dic[user]))
        for i in range(cur_test):
            interaction = sequence_dic[user].pop()
            new_data[interaction,2] = 1
    return new_data

In [ ]:
# remove user with less than 10 interactions
# for each user, divide first 90% interactions to train & last 10% of interactions to test
new_data = train_test_split_user(positive_interactions)

In [ ]:
# sanity check
user_id = 0

sample_trajectory = new_data[new_data[:,0] == user_id]
print("user_id {} - train ratio = {:.2f}, test ratio = {:.2f}".format(user_id,\
                                                                 (sample_trajectory[:,2] == 0).sum()/len(sample_trajectory),\
                                                                 (sample_trajectory[:,2] == 1).sum()/len(sample_trajectory)))

user_id 0 - train ratio = 0.90, test ratio = 0.10


In [ ]:
def sequence_generator(data, look_back = 50):
    train, test = [],[]
    unique_users = set(data[:,0])
    items_per_user = {int(user):[0 for i in range(look_back)] for user in unique_users}

    for (idx,row) in enumerate(data):
      user, item, train_test = row
      items_per_user[user] = items_per_user[user][1:]+[item+1]
      current_items = items_per_user[user]
      if train_test == 0:
        train.append([current_items[:-1],current_items[-1]])
      else:
        test.append([current_items[:-1],current_items[-1]])

    return train, test

In [ ]:
# convert to sequential data for sequential recommender system (look back = length of sequential time window)
look_back = 50
train, test = sequence_generator(new_data, look_back = look_back)

In [ ]:
train_num, train_data, train_labels = len(train), list(), list()
test_num, test_data, test_labels = len(test), list(), list()

for i in range(train_num):
    train_data.append(train[i][0])
    train_labels.append(train[i][1])

for i in range(test_num):
    test_data.append(test[i][0])
    test_labels.append(test[i][1])

In [ ]:
# class LSTM(nn.Module):
    # def __init__(self, num_items, emb_size, hidden_dim, n_layers=1):
#         torch.manual_seed(0)
#         np.random.seed(0)
#         super(LSTM, self).__init__()

#         #### YOUR CODE GOES HERE ####

#         #############################

#     def forward(self, input):

#         #### YOUR CODE GOES HERE ####

#         #############################

In [ ]:
item_emb_dim = 32
hidden_dim = 32

lstm_model = LSTM(num_items+1, item_emb_dim, hidden_dim).to(device)

In [ ]:
# loss function & experimental setting
criterion = None
optimizer = torch.optim.Adam(lstm_model.parameters())

batch_size = 1024
batch_num = len(train_data)//batch_size + 1
max_epoch = 500
patience = 10

In [ ]:
def mean_reciprocal_rank(pred, gt):
  probs = nn.functional.softmax(test_pred, dim=1)

  current_val = np.zeros((test_num,1))
  for i in range(test_num):
      current_test_label = test_labels[i]
      current_val[i] = probs[i,test_labels[i]]

  ranks = np.count_nonzero((probs - current_val)>0,axis=1)
  MRR = 0
  for i in range(test_num):
    MRR += 1/(ranks[i]+1)

In [25]:
max_mrr, best_epoch = 0, 0

for epoch in range(max_epoch):

  # batch training
  train_loss = 0
  lstm_model.train()
  for batch in range(batch_num):
    std, end = batch*batch_size, (batch+1)*batch_size
    end = train_num if (batch+1)*batch_size > train_num else (batch+1)*batch_size
    input, gt = train_data[std:end], train_labels[std:end]

    optimizer.zero_grad()
    input = torch.LongTensor(input).to(device)
    gt = torch.LongTensor(gt).to(device)

    pred = lstm_model(input)
    loss = criterion(pred,gt)
    train_loss += loss

    loss.backward()
    optimizer.step()

  # test evaluation
  lstm_model.eval()
  test_input = torch.LongTensor(test_data).to(device)
  test_pred = lstm_model(test_input)

  test_gt = torch.LongTensor(test_labels)
  MRR = mean_reciprocal_rank(test_pred.detach().cpu().numpy(), test_gt)

  MRR /= test_num
  if epoch % 10==0:
    print("epoch {} \ttrain_loss = {:.4f}\ttest_MRR = {:.4f}".format(epoch,train_loss,MRR))

  if max_mrr < MRR:
    max_mrr = MRR
    best_epoch = epoch
  elif epoch - best_epoch >= patience:
    print("Stop Training at epoch {} - test_MRR = {:.4f}".format(epoch, MRR))
    break

In [ ]:
import math
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=500):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        # pe = pe.unsqueeze(0).transpose(0, 1)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:,:x.size(1),:]
        return self.dropout(x)

In [ ]:
# class Transformer(nn.Module):
#     def __init__(self, num_items, item_emb_dim, hidden_dim, n_head=4, n_layers=1):
#         torch.manual_seed(0)
#         np.random.seed(0)
#         super(Transformer, self).__init__()

#         #### YOUR CODE GOES HERE ####

#         #############################

#     def forward(self, input):

#         #### YOUR CODE GOES HERE ####

#         #############################

In [ ]:
item_emb_dim = 32
hidden_dim = 32

transformer_model = Transformer(num_items+1, item_emb_dim, hidden_dim).to(device)

In [ ]:
# loss function & experimental setting
criterion = None
optimizer = torch.optim.Adam(transformer_model.parameters())

batch_size = 1024
batch_num = len(train_data)//batch_size + 1
max_epoch = 500
patience = 10

In [ ]:
max_mrr, best_epoch = 0, 0

for epoch in range(max_epoch):

  # batch training
  train_loss = 0
  transformer_model.train()
  for batch in range(batch_num):
    std, end = batch*batch_size, (batch+1)*batch_size
    end = train_num if (batch+1)*batch_size > train_num else (batch+1)*batch_size
    input, gt = train_data[std:end], train_labels[std:end]

    optimizer.zero_grad()
    input = torch.LongTensor(input).to(device)
    gt = torch.LongTensor(gt).to(device)

    pred = transformer_model(input)
    loss = criterion(pred,gt)
    train_loss += loss

    loss.backward()
    optimizer.step()

  # test evaluation
  transformer_model.eval()
  test_input = torch.LongTensor(test_data).to(device)
  test_pred = transformer_model(test_input)

  test_gt = torch.LongTensor(test_labels)
  MRR = mean_reciprocal_rank(test_pred.detach().cpu().numpy(), test_gt)

  MRR /= test_num
  if epoch % 10==0:
    print("epoch {} \ttrain_loss = {:.4f}\ttest_MRR = {:.4f}".format(epoch,train_loss,MRR))

  if max_mrr < MRR:
    max_mrr = MRR
    best_epoch = epoch
  elif epoch - best_epoch >= patience:
    print("Stop Training at epoch {} - test_MRR = {:.4f}".format(epoch, MRR))
    break

epoch 0 	train_loss = 621.2628	test_MRR = 0.0108
